# Import

In [1]:
from decouple import Config, RepositoryEnv
from ollama_interaction import embeding_ollama_request,generate_ollama_request

import pandas as pd
import plotly.express as px
import json

## Import des données sur les modèles
Nous importons les modèles servant de base pour VApp. Par défaut, les modèles sont quantifiés en int4.

In [2]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


In [3]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

## Import data base de travail
Import de la base de données générée dans generation-score.ipynb.

In [4]:
data_project_score = pd.read_csv("hard-database/data_project_scoring.csv").rename(columns={"scoing_made":"scoring_made"})
data_project_score['project_len_char'] = data_project_score['project_description'].apply(len)

In [5]:
data_project_score_select = data_project_score[data_project_score['project_score']>20]
data_project_score_select.head(3)

,Unnamed: 0,slug,url,name,name_initial,short_title,financers,financers_full,instructors,instructors_full,...,description_md,eligibility_md,token_numb_description_md,token_numb_description,token_numb_eligibility_md,token_numb_eligibility,project,project_score,scoring_made,project_len_char
146,143318,06ae-maitriser-le-volet-nuisances-sonores-dans...,/aides/06ae-maitriser-le-volet-nuisances-sonor...,Maîtriser le volet « nuisances sonores » dans ...,NaN,NaN,['Cerema'],"[{'id': 58, 'name': 'Cerema', 'logo': 'https:/...",[],[],...,\n**Description :**\n\n\n\n\n Le bruit routier...,\n**Public :** \n\n\n\n\n La formation s'adres...,718,920,150,182,Revitalisation d'une zone humide,23,5,32
401,162653,aider-les-commerces-a-se-moderniser,/aides/aider-les-commerces-a-se-moderniser/,Moderniser les commerces,Aide aux Commerces des Territoires (ACTe),NaN,['Communauté de communes des Falaises du Talou'],"[{'id': 2475, 'name': 'Communauté de communes ...",[],[],...,**Objectifs**\n\nLa Communauté de Communes\r\n...,Les entreprises doivent répondre\r\naux critèr...,1913,2568,670,1170,Revitalisation d'une zone humide,23,5,32
525,58670,eea7-appel-a-projets-0-phyto,/aides/eea7-appel-a-projets-0-phyto/,Réduire l'usage des produits phytosanitaires,NaN,NaN,"[""Conseil régional d'Occitanie""]","[{'id': 91, 'name': ""Conseil régional d'Occita...",[],[],...,\n Contexte et objectifs\n\n\n\n\n La région O...,Consultez la page de l'aide pour obtenir des d...,790,957,44,44,Revitalisation d'une zone humide,23,5,32


## Usage de la fonction gen_prompt_question_mono_aide

In [6]:
# Selection d'un échantillion simple
row = data_project_score_select[['description_md','eligibility_md','project_description']].iloc[0]

aide_description,aide_eligibility,project_description = row['description_md'],row['eligibility_md'],row['project_description']

### Exemple d'usage
Pour générer des résumés plus pertinents, nous avons besoin d'un LLM plus "intelligent". Notre choix s'est orienté vers Qwen 2.5-14B, qui représente un bon compromis entre vitesse, performance et intelligence.

In [7]:
# Fonction disponible sous ./prompt_script/gen_prompt_question_mono_aide.py
from prompt_script.gen_prompt_question_mono_aide import gen_prompt_question_mono_aide

In [8]:
model = "qwen2.5:14b"
model_options = model_data[model]

request_options = {
    "num_ctx": 16384,
    "num_predict": 512
}

num_question = 3


prompt_system, prompt_user = gen_prompt_question_mono_aide(aide_description,aide_eligibility,project_description,num_question)

response = generate_ollama_request(
    prompt_system=prompt_system,
    response_format="json",
    prompt_user=prompt_user,
    ollama_api_url=ollama_api_url,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    model=model, 
    request_options= request_options,  # Default to None
    seed=0,
    )

### Convertion au format json
**Attention** : lors de l'usage du json il peut rester des reliquats de balise html, un post traitement peux aider

In [9]:
json_text = response['response']

# Remplacer les caractères HTML par une apostrophe
json_text = json_text.replace('&#39;', "'")

# Convertir la chaîne en dictionnaire Python
response_json = json.loads(json_text)

response_json

{'Q1': "Pourriez-vous nous dire dans quelle mesure la revitalisation de cette zone humide nécessite une évaluation acoustique, et comment cela serait-il lié à l'infrastructure routière voisine?",
 'Q2': "Cette formation est-elle destinée aux personnes qui ont déjà suivi le module d'introduction ou pour celles qui n'en possèdent pas encore la connaissance nécessaire? Si c'est une personne spécifique, quel sera son niveau de connaissances en acoustique avant cette session?",
 'Q3': "Quels aspects pratiques spécifiques liés à l'étude du bruit attendez-vous d'apprendre lors des ateliers et études sur le terrain?"}

In [10]:
project_list = data_project_score_select.project.unique()

## Usage de la fonction gen_prompt_question_mono_aide

### Exemple d'usage
Nous avons remarqué que l'utilisation de plusieurs aides pour générer les questions était plus pertinente.
Nous avons donc implémenté une méthode capable d'intégrer trois questions basées sur un échantillon des trois meilleures aides trouvées.

Contrairement à la méthode "mono", nous n'intégrons pas l'éligibilité, ce qui permet d'avoir des questions plus générales et moins contraintes par les critères d'éligibilité. Cela aide l'utilisateur à mieux se concentrer sur son projet.

In [11]:
# Fonction disponible sous ./prompt_script/gen_prompt_question_multi_aide.py
from prompt_script.gen_prompt_question_multi_aide import gen_prompt_question_multi_aide

In [12]:
number_of_sub = 3

data_list_out = []

request_options = {
    "num_ctx": 16384,
    "num_predict": 512
}

for project_description in project_list:
    data_project_best = data_project_score_select[data_project_score_select['project']==project_description].sort_values("project_score")[-number_of_sub:]
    aide_description_list = data_project_best['description_md'].values
    sub1,sub2,sub3 = aide_description_list

    try:
        prompt_system, prompt_user = gen_prompt_question_multi_aide(aide_description_list,project_description,num_question)
        response = generate_ollama_request(
            prompt_system=prompt_system,
            response_format="json",
            prompt_user=prompt_user,
            ollama_api_url=ollama_api_url,
            bearer_token=ollama_bearer_token,
            model_options = model_options,  # Default to None
            model=model, 
            request_options= request_options,  # Default to None
            seed=0,
            )

        json_text = response['response']

        # Remplacer les caractères HTML par une apostrophe
        json_text = json_text.replace('&#39;', "'")

        # Convertir la chaîne en dictionnaire Python
        response_json = json.loads(json_text)

        q1 = response_json['Q1']
        q2 = response_json['Q2']
        q3 = response_json['Q3']

        data_list_out.append([project_description,sub1,sub2,sub3,q1,q2,q3])
        print('---------------')
        print(project_description)
    except Exception as error:
        print(error)

---------------
Revitalisation d'une zone humide
---------------
Entretient d'un vieux moulin
---------------
Réhabilitation d'une ancienne école en lieu dédié à la santé.
---------------
Voir fiche action PVD /ORT n°21 Après une première phase d’aménagement, la commune poursuit son action dans le cadre d’une 2ème phase d’aménagement. Cette prochaine étape concerne : - La mise en accessibilité du site aux personnes en situation de handicap et l’installation d’équipements en accès libre favorisant la pratique physique. - La protection de l’environnement par l’installation de sanitaires autonomes et la création d’un site O déchets - La création d'un parcours pédagogique
---------------
La commune dispose d'outils numériques qu'il est nécessaire d'optimiser et coordonner
---------------
Création d’un sentier thématique sur la forêt à Rieutord et sur le patrimoine à Usclades
---------------
Je souhaite refaire la voirie communal
---------------
Le projet consiste en l’aménagement d’un terr

In [13]:
data_question = pd.DataFrame(data_list_out,columns=['project','sub1','sub2','sub3','q1','q2','q3'])
data_question.to_csv('hard-database/data_question.csv')